# Obtención de datos mediante API

## Weather API

Es un proveedor de API de geolocalización y clima totalmente administrado que porporciona API extensas que van desde pronosticos meteorológicos y en tiempo real, clima historico, datos de calidad del aire, irradancia solar, solicitud masiva, busqueda de IP y astronomía.
[Weather API](https://www.weatherapi.com/)
- Clima en tiempo real
- Previsión meteorologica por horas hasta 14 días
- Clima futuro
- Intervalos diarios y horarios
- Datos de calidad del aire
- Irradiancia sola
- Datos de ubicación

Esta practica tiene como objetivo obtener datos que corresponden al clima de los 570 municipios del estado de Oaxaca.
Esta API es capaz de proporcionarnos toda esta ínformación de manera gratuita.

In [24]:
import os
import numpy as np
import pandas as pd
import requests
import time
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from unidecode import unidecode

## Scraping para obtener nombre de los municipios

Para asegurarnos que tenemos todos los nombres y que sean correctos, vamos a hacer una recolección de estos a través de una página del inegi que los lista.

Vamos a utilizar la herramienta Selenium para abrir un navegador, ir al enlace del inegi y guardar los nombres en una lista.

[Documentación completa de Selenium](https://pypi.org/project/selenium/)

In [25]:
# Set up the WebDriver
driver = webdriver.Chrome()

# Open the website
url = "https://cuentame.inegi.org.mx/monografias/informacion/oax/territorio/div_municipal.aspx?tema=me&e=20"
driver.get(url)

# Wait to url is ready
time.sleep(5)

# Read and save information
oax_municipalities = []
for i in range(1, 571):
    state = driver.find_element(
        By.XPATH, "//*[@id='keywords2']/tbody/tr[" + str(i) + "]/td[2]"
    )
    # Save and clean string
    oax_municipalities.append(unidecode(state.text))

# Close the browser
driver.quit()

El tiempo que toma obtener los datos de la API se vuelve considerablemente grande con el número de municipios que nosotros deseemamos consultar, para hacer pruebas rapidas la opción de `debug = True` es util pues solo trabajamos con 100 elementos para revisar que todo salga bien. Cuando estemos listos para obtener toda la información podemos poner la opción `debug = False`

In [26]:
debug = True
if debug:
    municipalities = oax_municipalities[300:400]
else:
    municipalities = oax_municipalities

## Envío de solicitudes a Weather API

Los datos que vamos a recolectar, nos lo van a proporcionar 3 request:
- search
- current
- astronomy

Se realizó una función que recibe un request y nos regresa una lista de objetos json con la información que nos proporciona la API.

Cada request llevará un nombre de municipio diferente, así que recorreremos la lista de los municipios y agregaremos los parametros `region:oaxaca` `country:mexico` para filtrar la información y así no obtendremos datos erroneos o de un lugar con el nombre similar.

In [27]:
def getApiData(endpoint, payload, municipalities, getFirst):
    data_collected = []
    for municipality in municipalities:
        payload["q"] = "'name':'" + municipality + "','region':'oaxaca','country':'mexico'"
        try:
            time.sleep(5)
            r = requests.get(url=endpoint, params=payload)
            if r.status_code == 200:
                json = r.json()
                if getFirst:
                    data_collected.append(json[0])
                else:
                    data_collected.append(json)
        except:
            continue
    return data_collected

Cargamos los datos de las variables de entorno del sistema

In [28]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

In [29]:
endpoint = "http://api.weatherapi.com/v1/"
payload = {"key": API_KEY}

In [30]:
search = "/search.json"

In [31]:
data_search = getApiData(
    endpoint=endpoint + search,
    payload=payload,
    municipalities=municipalities,
    getFirst=True,
)

In [32]:
current = "/current.json"

In [33]:
data_current = getApiData(
    endpoint=endpoint + current,
    payload=payload,
    municipalities=municipalities,
    getFirst=False,
)

In [34]:
astronomy = "/astronomy.json"

In [35]:
data_astronomy = getApiData(
    endpoint=endpoint + astronomy,
    payload=payload,
    municipalities=municipalities,
    getFirst=False,
)

## Normalización de datos Json y Conversion a DataFrames

Una vez que recolectamos los datos, los llevamos a dataframes

In [36]:

normalized_data_search = pd.json_normalize(data_search)
df_search_temp = pd.DataFrame.from_dict(normalized_data_search).add_prefix("location.")
df_search = df_search_temp.drop(["location.lat", "location.lon"], axis=1)
df_search.head()

,location.id,location.name,location.region,location.country,location.url
0,3209265,Abejones,Oaxaca,Mexico,abejones-oaxaca-mexico
1,3217399,Acatlan De Perez Figueroa,Oaxaca,Mexico,acatlan-de-perez-figueroa-oaxaca-mexico
2,3209199,Asuncion Cacalotepec,Oaxaca,Mexico,asuncion-cacalotepec-oaxaca-mexico
3,3209198,Asuncion Cuyotepeji,Oaxaca,Mexico,asuncion-cuyotepeji-oaxaca-mexico
4,3217411,Asuncion Ixtaltepec,Oaxaca,Mexico,asuncion-ixtaltepec-oaxaca-mexico


In [37]:
normalized_data_current = pd.json_normalize(data_current)
df_current = pd.DataFrame.from_dict(normalized_data_current)
df_current.head()

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,...,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Abejones,Oaxaca,Mexico,17.4364,-96.6085,America/Mexico_City,1736183029,2025-01-06 11:03,1736182800,2025-01-06 11:00,...,58.8,15.1,59.2,11.0,51.9,13.0,8.0,4.0,6.4,10.4
1,Acatlan De Perez Figueroa,Oaxaca,Mexico,18.5397,-96.6057,America/Mexico_City,1736183925,2025-01-06 11:18,1736183700,2025-01-06 11:15,...,71.7,23.1,73.5,20.2,68.3,10.0,6.0,5.0,3.6,5.8
2,Asuncion Cacalotepec,Oaxaca,Mexico,17.0361,-95.9550,America/Mexico_City,1736181473,2025-01-06 10:37,1736181000,2025-01-06 10:30,...,61.8,16.6,61.9,6.5,43.7,10.0,6.0,5.0,7.7,12.4
3,Asuncion Cuyotepeji,Oaxaca,Mexico,17.9167,-97.8000,America/Mexico_City,1736183940,2025-01-06 11:19,1736183700,2025-01-06 11:15,...,67.5,20.2,68.4,8.6,47.4,10.0,6.0,6.0,5.7,9.2
4,Asuncion Ixtaltepec,Oaxaca,Mexico,16.5031,-95.0608,America/Mexico_City,1736183877,2025-01-06 11:17,1736183700,2025-01-06 11:15,...,82.6,29.5,85.2,18.7,65.7,10.0,6.0,7.0,8.2,13.2


In [38]:
normalized_data_astronomy = pd.json_normalize(data_astronomy)
df_astronomy = pd.DataFrame.from_dict(normalized_data_astronomy)
df_astronomy.head()

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,astronomy.astro.sunrise,astronomy.astro.sunset,astronomy.astro.moonrise,astronomy.astro.moonset,astronomy.astro.moon_phase,astronomy.astro.moon_illumination,astronomy.astro.is_moon_up,astronomy.astro.is_sun_up
0,Abejones,Oaxaca,Mexico,17.4364,-96.6085,America/Mexico_City,1736187132,2025-01-06 12:12,06:58 AM,06:07 PM,12:00 PM,No moonset,First Quarter,42,0,1
1,Acatlan De Perez Figueroa,Oaxaca,Mexico,18.5397,-96.6057,America/Mexico_City,1736187073,2025-01-06 12:11,07:00 AM,06:05 PM,12:00 PM,No moonset,First Quarter,42,0,1
2,Asuncion Cacalotepec,Oaxaca,Mexico,17.0361,-95.9550,America/Mexico_City,1736187077,2025-01-06 12:11,06:55 AM,06:05 PM,11:58 AM,No moonset,First Quarter,42,0,1
3,Asuncion Cuyotepeji,Oaxaca,Mexico,17.9167,-97.8000,America/Mexico_City,1736187083,2025-01-06 12:11,07:04 AM,06:10 PM,12:05 PM,No moonset,First Quarter,42,0,1
4,Asuncion Ixtaltepec,Oaxaca,Mexico,16.5031,-95.0608,America/Mexico_City,1736187088,2025-01-06 12:11,06:50 AM,06:02 PM,11:55 AM,No moonset,First Quarter,42,0,1


## Unión de información

Como pudimos ver, los dataframes contienen diferentes columnas para cada municipio. 
Es necesario unirlas para tener todas las columnas en una sola tabla, debido a que hay columnas en común, utilizaremos la herramienta
merge para mezclar las tablas tomando como referencia el `name`, `region`, `country`, `lat`, `lon`, `tz_id` que son columnas que tienen en común y así se emparejan los datos.

In [39]:
df_astronomy_and_current = pd.merge(
    df_astronomy,
    df_current,
    on=[
        "location.name",
        "location.region",
        "location.country",
        "location.lat",
        "location.lon",
        "location.tz_id",
    ],
)
df_astronomy_and_current.head()

,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch_x,location.localtime_x,astronomy.astro.sunrise,astronomy.astro.sunset,...,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,Abejones,Oaxaca,Mexico,17.4364,-96.6085,America/Mexico_City,1736187132,2025-01-06 12:12,06:58 AM,06:07 PM,...,58.8,15.1,59.2,11.0,51.9,13.0,8.0,4.0,6.4,10.4
1,Acatlan De Perez Figueroa,Oaxaca,Mexico,18.5397,-96.6057,America/Mexico_City,1736187073,2025-01-06 12:11,07:00 AM,06:05 PM,...,71.7,23.1,73.5,20.2,68.3,10.0,6.0,5.0,3.6,5.8
2,Asuncion Cacalotepec,Oaxaca,Mexico,17.0361,-95.9550,America/Mexico_City,1736187077,2025-01-06 12:11,06:55 AM,06:05 PM,...,61.8,16.6,61.9,6.5,43.7,10.0,6.0,5.0,7.7,12.4
3,Asuncion Cuyotepeji,Oaxaca,Mexico,17.9167,-97.8000,America/Mexico_City,1736187083,2025-01-06 12:11,07:04 AM,06:10 PM,...,67.5,20.2,68.4,8.6,47.4,10.0,6.0,6.0,5.7,9.2
4,Asuncion Ixtaltepec,Oaxaca,Mexico,16.5031,-95.0608,America/Mexico_City,1736187088,2025-01-06 12:11,06:50 AM,06:02 PM,...,82.6,29.5,85.2,18.7,65.7,10.0,6.0,7.0,8.2,13.2


El conjunto de datos `search` contiene un elemento muy importante que es el `id` de cada municipio, entonces vamos a mezclar la tabla que ya fue unida anteriormente con los datos de search tomando como referencias las columnas con el mismo nombre.

In [40]:
df_full = pd.merge(
    df_search,
    df_astronomy_and_current,
    on=[
        "location.name",
        "location.region",
        "location.country",
    ],
)

In [56]:
df_full.head()

,location.id,location.name,location.region,location.country,location.url,location.lat,location.lon,location.tz_id,location.localtime_epoch_x,location.localtime_x,...,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph
0,3209265,Abejones,Oaxaca,Mexico,abejones-oaxaca-mexico,17.4364,-96.6085,America/Mexico_City,1736187132,2025-01-06 12:12,...,58.8,15.1,59.2,11.0,51.9,13.0,8.0,4.0,6.4,10.4
1,3217399,Acatlan De Perez Figueroa,Oaxaca,Mexico,acatlan-de-perez-figueroa-oaxaca-mexico,18.5397,-96.6057,America/Mexico_City,1736187073,2025-01-06 12:11,...,71.7,23.1,73.5,20.2,68.3,10.0,6.0,5.0,3.6,5.8
2,3209199,Asuncion Cacalotepec,Oaxaca,Mexico,asuncion-cacalotepec-oaxaca-mexico,17.0361,-95.9550,America/Mexico_City,1736187077,2025-01-06 12:11,...,61.8,16.6,61.9,6.5,43.7,10.0,6.0,5.0,7.7,12.4
3,3209198,Asuncion Cuyotepeji,Oaxaca,Mexico,asuncion-cuyotepeji-oaxaca-mexico,17.9167,-97.8000,America/Mexico_City,1736187083,2025-01-06 12:11,...,67.5,20.2,68.4,8.6,47.4,10.0,6.0,6.0,5.7,9.2
4,3217411,Asuncion Ixtaltepec,Oaxaca,Mexico,asuncion-ixtaltepec-oaxaca-mexico,16.5031,-95.0608,America/Mexico_City,1736187088,2025-01-06 12:11,...,82.6,29.5,85.2,18.7,65.7,10.0,6.0,7.0,8.2,13.2


In [57]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 540 entries, 0 to 834
Data columns (total 51 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   location.id                        540 non-null    int64  
 1   location.name                      540 non-null    object 
 2   location.region                    540 non-null    object 
 3   location.country                   540 non-null    object 
 4   location.url                       540 non-null    object 
 5   location.lat                       540 non-null    float64
 6   location.lon                       540 non-null    float64
 7   location.tz_id                     540 non-null    object 
 8   location.localtime_epoch_x         540 non-null    int64  
 9   location.localtime_x               540 non-null    object 
 10  astronomy.astro.sunrise            540 non-null    object 
 11  astronomy.astro.sunset             540 non-null    object 
 12 

In [72]:
df_full = df_full.drop_duplicates(subset="location.id").reset_index(drop=True)

## Exportación de datos

Una vez que los datos están en una sola tabla, lo guardaremos con el formato csv

In [75]:
df_full.to_csv("dataset/dataset_raw/weather.csv")

El archivo csv se encuentra en el siguiente enlace [aqui](https://drive.google.com/file/d/163nK4yqm8U4i1UITlCHTWCwWWM_8LiIU/view?usp=sharing)